In [1]:
#get access google drive data into google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/dataset_new.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')

finish


In [3]:
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

In [4]:
### Creating model and layers
model = Sequential();   ## blank model
# Step 1 - Convolution
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))   ## applied for under/over fitting
# Step 1 - Convolution
model.add(Conv2D(64,(3,3),activation='relu'))
# Step 2 - Pooling
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
# Step 2 - Pooling
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
# Step 3 - Flattening
model.add(Flatten())
### Hidden and output
# Step 4 - Full connection
#model.add(Dense(128, activation='relu'))    ### hidden layer
model.add(Dense(1, activation='sigmoid'))   ## output layer
# Compiling the CNN
model.compile(optimizer ='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [5]:
# https://keras.io/api/preprocessing/image/#imagedatagenerator-class

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [6]:
training_set = train_datagen.flow_from_directory('/content/dataset_new/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory('/content/dataset_new/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [7]:
model.fit(training_set,
          epochs = 200,
          steps_per_epoch=2000//32,
          validation_steps=500//32,
          validation_data = test_set)

Epoch 1/200
62/62 [==============================] - 41s 143ms/step - loss: 0.7009 - accuracy: 0.5056 - val_loss: 0.6925 - val_accuracy: 0.5292
Epoch 2/200
62/62 [==============================] - 8s 137ms/step - loss: 0.6907 - accuracy: 0.5380 - val_loss: 0.6869 - val_accuracy: 0.5875
Epoch 3/200
62/62 [==============================] - 8s 137ms/step - loss: 0.6764 - accuracy: 0.5973 - val_loss: 0.6504 - val_accuracy: 0.6521
Epoch 4/200
62/62 [==============================] - 8s 135ms/step - loss: 0.6478 - accuracy: 0.6520 - val_loss: 0.6291 - val_accuracy: 0.7000
Epoch 5/200
62/62 [==============================] - 8s 136ms/step - loss: 0.6205 - accuracy: 0.6744 - val_loss: 0.6347 - val_accuracy: 0.6583
Epoch 6/200
62/62 [==============================] - 8s 137ms/step - loss: 0.6160 - accuracy: 0.6750 - val_loss: 0.6760 - val_accuracy: 0.5896
Epoch 7/200
62/62 [==============================] - 8s 135ms/step - loss: 0.6147 - accuracy: 0.6580 - val_loss: 0.6524 - val_accuracy: 0.618

In [27]:
import numpy as np
from keras.preprocessing import image 
test_image = image.load_img('/content/dog_new.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)  
if result[0][0] == 1:
  prediction = 'dog'
  print(prediction)
else:
  prediction = 'cat'
  print(prediction)

dog


In [ ]:
len(test_image)

In [28]:
### Saving the model
model.save("cat_dog_cnn.h5")

In [29]:
### load the model
import keras as tf
mymodel = keras.models.load_model("/content/cat_dog_cnn.h5")